In [25]:
import sys

In [26]:
sys.path.append("..")

In [27]:
from elastic_function import elastic_search
from openai import OpenAI

In [28]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [29]:
query = 'the course has already started, can I still enroll?'

In [30]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt, model):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [31]:
def rag(query, model="phi3"):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, model)
    return answer

In [32]:
rag(query,'gemma:2b')

'No, the passage does not indicate that it is possible to enroll in the course after it has started.'

In [14]:
rag(query,'gemma:2b')

'No, the course cannot be enrolled after the start date. According to the context, you are eligible to submit homeworks and participate in the course, but you will not be able to submit some homeworks or complete the final projects.'

In [16]:
rag(query,'phi3')

' Yes, according to our FAQ database: "Yes, but it\'s advised not to leave everything until last minute as there are deadlines for turning in final projects." However, while working on two out of three course project options by the end of November will still give you eligibility for a certificate.\n'